In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [6]:
ls

data/  day4.ipynb  day4_meta.ipynb  LICENSE  matrix_one/  README.md


In [7]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [22]:
df[ 'brand_cat' ] = df[ 'brand' ].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [23]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [25]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [31]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [46]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [47]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [50]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [51]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_ul safety listing', 'feat_alarm', 'feat_fit:', 'feat_year made',
       'feat_accessory type', 'feat_safety apparel product type',
       'feat_country/ organization', 'feat_item character',
       'feat_fits most screen size', 'feat_best sellers rank'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [60]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df[ 'feat_brand_cat' ] = df[ 'feat_brand' ].factorize()[0]
df[ 'feat_color_cat' ] = df[ 'feat_color' ].factorize()[0]
df[ 'feat_gender_cat' ] = df[ 'feat_gender' ].factorize()[0]
df[ 'feat_manufacturer part number_cat' ] = df[ 'feat_manufacturer part number' ].factorize()[0]
df[ 'feat_material_cat' ] = df[ 'feat_material' ].factorize()[0]

df[ 'feat_sport_cat' ] = df[ 'feat_sport' ].factorize()[0]
df[ 'feat_style_cat' ] = df[ 'feat_style' ].factorize()[0]

for key in keys:
  df[ get_name_feat(key) + '_cat' ] = df[get_name_feat(key)].factorize()[0]

  


In [73]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']


In [75]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.26947117316304, 4.21511835127189)

In [99]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_certifications and listings',
 'feat_fabrication',
 'feat_multi pack indicator',
 'feat_recommended location',
 'feat_location - city/state',
 'feat_catalog',
 'feat_shoe category',
 'feat_clothing category',
 'feat_location - country',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_size/dimensions_cat',
 'feat_display technology_cat',
 'feat_shape_cat',
 'feat_bridge/temple size:_cat',
 'feat_sock style_cat',
 'feat_number of batteries included_cat',
 'feat_item weight unit of measure_cat',
 'feat_adidas_cat',
 'feat_mpn#_cat',
 'feat_navy blue_cat',
 'feat_shoe closure_cat',
 'feat_batteries required?_cat',
 'feat_band material_cat',
 'feat_age gender group_cat',
 'feat_band length (in.)_cat',
 'feat_hunting_cat',
 'feat_carats_cat',
 'feat_fits model_cat',
 'feat_inv_cat',
 'feat_airport friendly_cat',
 'feat_product in inches (l x w

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
feats = ['brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat',  'feat_style_cat', 'feat_sport_cat']
# feats += feats_cat
# feats = list(set(feats))

result = run_model(feats, model)

In [116]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-57.3243848977688, 4.381208055632062)


Weight,Feature
0.2557 ± 0.0080,brand_cat
0.1032 ± 0.0075,feat_material_cat
0.0252 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0091 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [84]:
df['brand'].value_counts(normalize=True)

nike            0.097210
puma            0.033315
ralph lauren    0.028775
vans            0.021116
new balance     0.020295
                  ...   
sessions        0.000055
sojourns end    0.000055
big bear        0.000055
fits            0.000055
nitrogen        0.000055
Name: brand, Length: 1732, dtype: float64

In [96]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'football', 'condition': 'new without box', 'type': 'cleats'},
       {'sport': 'soccer', 'material': 'synthetic', 'type': 'cleats', 'condition': 'new with box'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'style': 'basketball shoes', 'country/region of manufacture': 'vietnam', 'type': 'athletic-inspired', 'condition': 'new with box', 'pattern': 'solid', 'material': 'suede'},
       {'type': 'cleats', 'condition': 'new with box'}], dtype=object)

In [97]:
df[ 'feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
mens                   4
toddler                4
infant                 4
boys'                  3
men||women             2
women                  2
youth                  2
women ,�� unisex       2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [118]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   .gitignore
	modified:   day4.ipynb
	deleted:    matrix_one (1)/day3.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	day4_meta.ipynb
	matrix_one/

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git add da